In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
data['education'] = data['education'].str.lower()
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)
data['days_employed'] = data['days_employed'].astype(int)
data['total_income'] = data['total_income'].astype(int)
data['gender'] = data['gender'].replace('XNA', 'Nan')
data = data.drop_duplicates().reset_index(drop=True)
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [4]:
def children_group(children):
    if children == -1 or children == 20:
        return 'подозрительные значения'
    if 1 <= children <= 2:
        return 'обычная семья'
    if children > 2:
        return 'многодетная семья'
    if children == 0:
        return 'бездетная семья'
data['children_group'] = data['children'].apply(children_group)
Child_group = data.groupby('children_group')['debt'].count()
Child_group

children_group
бездетная семья            14091
многодетная семья            380
обычная семья               6860
подозрительные значения      123
Name: debt, dtype: int64

Я думала на счет значения -1. Два из наиболее вероятных варианта: люди отмечали своих умерших детей (ну почему бы и нет...) или это тире и косяк человека, кто готовил данные. Но во-первых, их многовато для того, чтобы считать случайной ошибкой, а во-вторых, 1 и - стоят в разных концах клавиатуры. Как?!
Конечно, скорее всего это ошибка данных и нужно заменить значения на 1 с помощью replace.

In [5]:
# проверим теорию, что положительные числа отработанных дней только у пенсионеров
# похоже, что да...
def days_retired(row):
    days = row['days_employed']
    income = row['income_type']
    if days >= 0:
        if income == 'пенсионер':
            return 'True'
    return 'False'
data['false/true'] = data.apply(days_retired, axis = 1)
print(data['false/true'].value_counts())
print(data[data['income_type'] == 'пенсионер']['income_type'].count())

False    17625
True      3829
Name: false/true, dtype: int64
3829


In [6]:
from pymystem3 import Mystem
m = Mystem()
data['purpose_lem'] = data['purpose'].apply(m.lemmatize)
data['purpose_lem'] = data['purpose_lem'].apply(' '.join)
lemmas = data['purpose_lem'].unique()
from collections import Counter
Counter(lemmas)

Counter({'покупка   жилье \n': 1,
         'приобретение   автомобиль \n': 1,
         'дополнительный   образование \n': 1,
         'сыграть   свадьба \n': 1,
         'операция   с   жилье \n': 1,
         'образование \n': 1,
         'на   проведение   свадьба \n': 1,
         'покупка   жилье   для   семья \n': 1,
         'покупка   недвижимость \n': 1,
         'покупка   коммерческий   недвижимость \n': 1,
         'покупка   жилой   недвижимость \n': 1,
         'строительство   собственный   недвижимость \n': 1,
         'недвижимость \n': 1,
         'строительство   недвижимость \n': 1,
         'на   покупка   подержать   автомобиль \n': 1,
         'на   покупка   свой   автомобиль \n': 1,
         'операция   с   коммерческий   недвижимость \n': 1,
         'строительство   жилой   недвижимость \n': 1,
         'жилье \n': 1,
         'операция   со   свой   недвижимость \n': 1,
         'автомобиль \n': 1,
         'заниматься   образование \n': 1,
         'сделка   с

### Задание 1. 

Пропущенные значения появились в столбцах days_employed и total_income. Проверив заполнением и удалением лишних нулей, выяснилось, что они находились в одинаковых строках. Их отсутствие возможно объясняется нежеланием людей среднего возраста, со средним образованием и без детей (превалирующих в данных с пропущенными значениями) раскрывать свои доходы и стаж работы.

In [7]:
# Есть ли зависимость между наличием детей и возвратом кредита в срок?

depend_ch = data.groupby('children')['debt'].sum()
data_children = data['children'].value_counts()
new = pd.merge(data_children, depend_ch, right_on='children', left_index=True)
# это единственный вариант, который сработал. Не знаю, как сгруппировать по формуле, данной в теории в нашем курсе
new = new.set_axis(['children_count', 'debt'], axis=1, inplace=False)
new['debtors'] = new['debt'] / new['children_count']
new['debtors'] = new['debtors'].map(lambda n: '{:,.2%}'.format(n))
new = new.sort_values(by='children', ascending=True)
new

,children_count,debt,debtors
children,,,
-1,47,1,2.13%
0,14091,1063,7.54%
1,4808,444,9.23%
2,2052,194,9.45%
3,330,27,8.18%
4,41,4,9.76%
5,9,0,0.00%
20,76,8,10.53%


По данной таблице можно сделать вывод, что количество детей очевидно влияет на задолженность по кредиту. Может, чем меньше детей, тем проще отдавать кредит? Явно дети ответственности перед банком не добавляют.

In [8]:
# Есть ли зависимость между семейным положением и возвратом кредита в срок?

depend_fam = data.groupby('family_status')['debt'].sum()
data_fam = data['family_status'].value_counts()
new_f = pd.merge(data_fam, depend_fam, right_on='family_status', left_index=True)
new_f = new_f.set_axis(['Family_count', 'debt'], axis=1, inplace=False)
new_f['debtors_F'] = new_f['debt'] / new_f['Family_count']
new_f['debtors_F'] = new_f['debtors_F'].map(lambda n: '{:,.2%}'.format(n))
new_f = new_f.sort_values(by='debtors_F', ascending=True)
new_f

,Family_count,debt,debtors_F
family_status,,,
вдовец / вдова,959,63,6.57%
в разводе,1195,85,7.11%
женат / замужем,12339,931,7.55%
гражданский брак,4151,388,9.35%
Не женат / не замужем,2810,274,9.75%


Да, брак очевидно положительно повлиял на ответственность. Люди, прошедшие эту жизненную школу, явно лучше отдают долги.

In [9]:
# Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
#from statistics import median
#print(data['total_income'].max())
#print(data[data['total_income'] != 0]['total_income'].min())
#print(data[data['total_income'] != 0]['total_income'].median())
#print(median([20667, 145017.0]))
#print(median([145017.0, 2265604]))

def total_income_group(salary):
    if 82842 > salary :
        return 'мало зарабатывает'
    if 82842 <= salary < 145017:
        return 'средний заработок'  
    if 145017 <= salary < 1205310:
        return 'средний заработок'
    if 1205310 <= salary:
        return 'заработок сверх нормы'
data['salary'] = data['total_income'].apply(total_income_group)
Salary = data.groupby('salary')['debt'].sum()

depend_income = data.groupby('salary')['debt'].sum()
data_income = data['salary'].value_counts()
new_in = pd.merge(data_income, depend_income, right_on='salary', left_index=True)
new_in = new_in.set_axis(['Salary_count', 'debt'], axis=1, inplace=False)
new_in['debtors_F'] = new_in['debt'] / new_in['Salary_count']
new_in['debtors_F'] = new_in['debtors_F'].map(lambda n: '{:,.2%}'.format(n))
new_in = new_in.sort_values(by='debtors_F', ascending=True)
new_in

,Salary_count,debt,debtors_F
salary,,,
заработок сверх нормы,13,1,7.69%
мало зарабатывает,4674,367,7.85%
средний заработок,16767,1373,8.19%


Средний уровень заработка немного отстает по качеству закрытия кредитов от других уровней заработка.

In [10]:
# Как разные цели кредита влияют на его возврат в срок?
#print(data['purpose_lem'].value_counts())
def purpose_group(mean):
    if 'автомобиль' in mean:
        return 'покупка автомобиля'
    if 'свадьба' in mean:
        return 'оплата свадьбы'
    if ('недвижимость ' in mean) or ('жилье' in mean):
        return 'покупка жилья'
    if 'образование' in mean:
        return 'оплата образования'
data['purpose_lem_new'] = data['purpose_lem'].apply(purpose_group)

purpose = data.groupby('purpose_lem_new')['debt'].sum()
data_purpose = data['purpose_lem_new'].value_counts()
new_pur = pd.merge(data_purpose, purpose, right_on='purpose_lem_new', left_index=True)
new_pur = new_pur.set_axis(['Purpose_count', 'debt'], axis=1, inplace=False)
new_pur['debtors_P'] = new_pur['debt'] / new_pur['Purpose_count']
new_pur['debtors_P'] = new_pur['debtors_P'].map(lambda n: '{:,.2%}'.format(n))
new_pur = new_pur.sort_values(by='debtors_P', ascending=True)
new_pur

,Purpose_count,debt,debtors_P
purpose_lem_new,,,
покупка жилья,10811,782,7.23%
оплата свадьбы,2324,186,8.00%
оплата образования,4013,370,9.22%
покупка автомобиля,4306,403,9.36%


Жильем народ дорожит больше, чем автомобилем. Скорее всего объясняется долгосрочностью и дороговизной кредита. 

## Задание 2. Общий вывод
Проведя анализ по некоторым данным в таблице, можно сказать, что уровень должников в принципе остается средним 8-9%. На общем фоне выделяются семьи с -1 ребенком. не понятно, это либо ошибка, либо действительно потерянный в семье человек. Странно, что семьи с 5 детьми имеют меньше задолженностей, чем люди с (очень) повышенным достатком. Хотя там и выборка не такая большая. 

Наиболее добросовестными плательщиками оказались бездетные, овдовевшие ипотечники, зарабатывающие выше среднего.